# Train unimodal CT

In [1]:
!git clone https://github.com/PietroCaforio/research-biocv-proj
!cd research-biocv-proj && git switch dev

Cloning into 'research-biocv-proj'...
remote: Enumerating objects: 172, done.
remote: Counting objects: 100% (172/172), done.
remote: Compressing objects: 100% (128/128), done.
remote: Total 172 (delta 88), reused 109 (delta 36), pack-reused 0 (from 0)
Receiving objects: 100% (172/172), 3.39 MiB | 10.21 MiB/s, done.
Resolving deltas: 100% (88/88), done.
Branch 'dev' set up to track remote branch 'dev' from 'origin'.
Switched to a new branch 'dev'


In [2]:
!cd research-biocv-proj && git pull

Already up to date.


In [3]:
!pip install wandb

In [4]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("wandb_api_key")

In [5]:
import wandb
wandb.login(key=secret_value_0)

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [6]:
import sys
from pathlib import Path

# Add the 'data' directory to sys.path
sys.path.append(str(Path('research-biocv-proj').resolve()))
from data.unimodal import *
from pathlib import Path

import numpy as np
import torch
from torch.utils.data import DataLoader

### Train ResNet model

In [7]:
def train(model,config, run_name=None):
  wandb.init(
    # set the wandb project where this run will be logged
    project="unimodal_ct_training",
    name = run_name,
    # track hyperparameters and run metadata
    config=config
  )
  optimizer = optim.Adam(model.parameters(), lr=config["learning_rate"], weight_decay=config["weight_decay"])
  criterion = nn.CrossEntropyLoss()
  scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min',factor = config["reduce_lr_factor"], patience = config["patience"])
  # Training loop
  num_epochs = config["epochs"]
  for epoch in range(num_epochs):
      model.train()
      running_loss = 0.0

      for batch in train_loader:
          frames = batch['frame'].float().to(device)
          labels = batch['label'].long().to(device)

          optimizer.zero_grad()
          outputs = model(frames)
          loss = criterion(outputs.logits, labels)

          loss.backward()
          optimizer.step()

          running_loss += loss.item()

      print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}")

      # Validation loop
      model.eval()
      val_loss = 0.0
      correct = 0
      total = 0
      # Initialize counters for each class (G1, G2, G3)
      correct_per_class = [0, 0, 0]  # For G1, G2, G3
      total_per_class = [0, 0, 0]  # For G1, G2, G3

      with torch.no_grad():
          for batch in val_loader:
              frames = batch['frame'].float().to(device)
              labels = batch['label'].long().to(device)

              outputs = model(frames)
              loss = criterion(outputs.logits, labels)

              val_loss += loss.item()
              _, predicted = torch.max(outputs.logits, 1)
              total += labels.size(0)
              correct += (predicted == labels).sum().item()

              # Calculate accuracy per class
              for i in range(3):  # We have 3 classes: G1 (0), G2 (1), G3 (2)
                  correct_per_class[i] += ((predicted == i) & (labels == i)).sum().item()
                  total_per_class[i] += (labels == i).sum().item()
      scheduler.step(val_loss)
      # Compute total accuracy and per-class accuracy
      total_accuracy = 100 * correct / total
      class_accuracy = [(100 * correct_per_class[i] / total_per_class[i]) if total_per_class[i] > 0 else 0 for i in range(3)]
      print(f"Validation Loss: {val_loss/len(val_loader)}, Total Accuracy: {total_accuracy:.2f}%")
      print(f"Accuracy per class - G1: {class_accuracy[0]:.2f}%, G2: {class_accuracy[1]:.2f}%, G3: {class_accuracy[2]:.2f}%")
      # log metrics to wandb
      wandb.log({"Total Accuracy": total_accuracy, "Validation Loss": val_loss/len(val_loader), "G1_Acc":class_accuracy[0], "G2_Acc":class_accuracy[1], "G3_Acc":class_accuracy[2]})
  wandb.finish()  

In [8]:
import torch.nn as nn
import torch.optim as optim
from transformers import ResNetForImageClassification

In [9]:
train_dataset = UnimodalCTDataset(split='train',dataset_path = "/kaggle/input/preprocessed57patientscptacpda/processed/" )
val_dataset = UnimodalCTDataset(split='val',dataset_path = "/kaggle/input/preprocessed57patientscptacpda/processed/")

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [10]:
print(f"Training set stats:{train_dataset.stats()}")
print(f"Validation set stats:{val_dataset.stats()}")

Training set stats:{'length': 2292, 'class_frequency': {'G1': 78, 'G2': 1543, 'G3': 671}}
Validation set stats:{'length': 335, 'class_frequency': {'G1': 37, 'G2': 166, 'G3': 132}}


### Resnet-50

In [11]:
model = ResNetForImageClassification.from_pretrained('microsoft/resnet-50')
model.classifier[-1] = nn.Linear(model.classifier[-1].in_features, UnimodalCTDataset.num_classes) #Adjusting the final layer to the unimodal number of classes

config.json:   0%|          | 0.00/69.6k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/102M [00:00<?, ?B/s]

In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

ResNetForImageClassification(
  (resnet): ResNetModel(
    (embedder): ResNetEmbeddings(
      (embedder): ResNetConvLayer(
        (convolution): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        (normalization): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (activation): ReLU()
      )
      (pooler): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    )
    (encoder): ResNetEncoder(
      (stages): ModuleList(
        (0): ResNetStage(
          (layers): Sequential(
            (0): ResNetBottleNeckLayer(
              (shortcut): ResNetShortCut(
                (convolution): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
                (normalization): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              )
              (layer): Sequential(
                (0): ResNetConvLayer(
                  (convolution): Conv2d(64

In [13]:
config={
    "learning_rate": 0.0001,
    "architecture": "microsoft/resnet-34",
    "epochs": 70,
    "weight_decay": 0.01,
    "reduce_lr_factor": 0.25,
    "patience": 8
    }
train(model, config, run_name = config["architecture"])

wandb: Currently logged in as: pietro-caforio (pietro-caforio-politecnico-di-milano). Use `wandb login --relogin` to force relogin
wandb: wandb version 0.18.0 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.17.7
wandb: Run data is saved locally in /kaggle/working/wandb/run-20240916_111032-dommbblu
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run microsoft/resnet-34
wandb: ⭐️ View project at https://wandb.ai/pietro-caforio-politecnico-di-milano/unimodal_ct_training
wandb: 🚀 View run at https://wandb.ai/pietro-caforio-politecnico-di-milano/unimodal_ct_training/runs/dommbblu


Epoch 1, Loss: 0.8967649390300115
Validation Loss: 13.800668174570257, Total Accuracy: 49.55%
Accuracy per class - G1: 0.00%, G2: 100.00%, G3: 0.00%
Epoch 2, Loss: 0.6648679015537103
Validation Loss: 42.70067577741363, Total Accuracy: 49.55%
Accuracy per class - G1: 0.00%, G2: 100.00%, G3: 0.00%
Epoch 3, Loss: 0.5311581922902001
Validation Loss: 54.21851275861263, Total Accuracy: 49.55%
Accuracy per class - G1: 0.00%, G2: 100.00%, G3: 0.00%
Epoch 4, Loss: 0.4109079742597209
Validation Loss: 69.94051199338652, Total Accuracy: 53.73%
Accuracy per class - G1: 0.00%, G2: 100.00%, G3: 10.61%
Epoch 5, Loss: 0.30273878367410767
Validation Loss: 86.66379305855794, Total Accuracy: 53.13%
Accuracy per class - G1: 0.00%, G2: 100.00%, G3: 9.09%
Epoch 6, Loss: 0.23817062812546888
Validation Loss: 107.91089578921145, Total Accuracy: 55.52%
Accuracy per class - G1: 0.00%, G2: 98.80%, G3: 16.67%
Epoch 7, Loss: 0.19404704651484886
Validation Loss: 125.61574527621269, Total Accuracy: 74.03%
Accuracy per

wandb:                                                                                
wandb: 
wandb: Run history:
wandb:          G1_Acc ▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
wandb:          G2_Acc ███▆▅█▂▁▆▄▇▃▄▄▄▃▃▅▃▅▄▄▂▅▄▃▄▄▄▄▄▄▅▂▃▄▄▃▄▃
wandb:          G3_Acc ▁▁▂▃▇▆▇███▆█▇███▇▇▇▇▇██▇██▇█▇▇█▇█▇████▇█
wandb:  Total Accuracy ▁▁▂▂▇▆▇▇██▆▇▇██▇▇▇▇▇▇██▇▇█▇█▇▇█▇█▇███▇▇█
wandb: Validation Loss ▁▂▃▄▆▆▇▇▇███▇█▇▇▇█▇▇▇█▇██▇▇█▇▇█▇█▇▇▇▇▇▇█
wandb: 
wandb: Run summary:
wandb:          G1_Acc 0.0
wandb:          G2_Acc 96.98795
wandb:          G3_Acc 74.24242
wandb:  Total Accuracy 77.31343
wandb: Validation Loss 213.80878
wandb: 
wandb: 🚀 View run microsoft/resnet-34 at: https://wandb.ai/pietro-caforio-politecnico-di-milano/unimodal_ct_training/runs/dommbblu
wandb: ⭐️ View project at: https://wandb.ai/pietro-caforio-politecnico-di-milano/unimodal_ct_training
wandb: Synced 5 W&B file(s), 0 media file(s), 0 artifact file(s) and 0 other file(s)
wandb: Find logs at: ./wandb/run-20240916_111032-dommbb

### Resnet-18

In [14]:
model = ResNetForImageClassification.from_pretrained('microsoft/resnet-18')
model.classifier[-1] = nn.Linear(model.classifier[-1].in_features, UnimodalCTDataset.num_classes) #Adjusting the final layer to the unimodal number of classes

config.json:   0%|          | 0.00/69.5k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/46.8M [00:00<?, ?B/s]

In [15]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

ResNetForImageClassification(
  (resnet): ResNetModel(
    (embedder): ResNetEmbeddings(
      (embedder): ResNetConvLayer(
        (convolution): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        (normalization): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (activation): ReLU()
      )
      (pooler): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    )
    (encoder): ResNetEncoder(
      (stages): ModuleList(
        (0): ResNetStage(
          (layers): Sequential(
            (0): ResNetBasicLayer(
              (shortcut): Identity()
              (layer): Sequential(
                (0): ResNetConvLayer(
                  (convolution): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
                  (normalization): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
                  (activation): ReLU()
           

In [16]:
config={
    "learning_rate": 0.0001,
    "architecture": "microsoft/resnet-18",
    "epochs": 70,
    "weight_decay": 0.0,
    "reduce_lr_factor": 0.25,
    "patience": 20
    }
train(model, config, run_name = config["architecture"])

wandb: wandb version 0.18.0 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.17.7
wandb: Run data is saved locally in /kaggle/working/wandb/run-20240916_113810-6m6gmdvs
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run microsoft/resnet-18
wandb: ⭐️ View project at https://wandb.ai/pietro-caforio-politecnico-di-milano/unimodal_ct_training
wandb: 🚀 View run at https://wandb.ai/pietro-caforio-politecnico-di-milano/unimodal_ct_training/runs/6m6gmdvs


Epoch 1, Loss: 0.12301912143852355
Validation Loss: 1.612691484074193, Total Accuracy: 58.51%
Accuracy per class - G1: 0.00%, G2: 78.92%, G3: 49.24%
Epoch 2, Loss: 0.00263932763482444
Validation Loss: 1.974825443202545, Total Accuracy: 58.81%
Accuracy per class - G1: 0.00%, G2: 75.30%, G3: 54.55%
Epoch 3, Loss: 0.0013412428969685505
Validation Loss: 2.0486556853919122, Total Accuracy: 58.81%
Accuracy per class - G1: 0.00%, G2: 75.30%, G3: 54.55%
Epoch 4, Loss: 0.0008917585251361339
Validation Loss: 2.119332629811569, Total Accuracy: 60.00%
Accuracy per class - G1: 0.00%, G2: 75.30%, G3: 57.58%
Epoch 5, Loss: 0.0006831847080724805
Validation Loss: 2.2037898544116286, Total Accuracy: 59.40%
Accuracy per class - G1: 0.00%, G2: 75.30%, G3: 56.06%
Epoch 6, Loss: 0.0008047883796583241
Validation Loss: 2.3324454007299873, Total Accuracy: 60.00%
Accuracy per class - G1: 0.00%, G2: 75.30%, G3: 57.58%
Epoch 7, Loss: 0.0005632584181714467
Validation Loss: 2.372493521876591, Total Accuracy: 57.01%

wandb:                                                                                
wandb: 
wandb: Run history:
wandb:          G1_Acc ▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
wandb:          G2_Acc ▅▁▁▁▁▂▄▁▃▄█▆▂▁▄▂▁▂▄▂▂▂▂▂▂▁▂▁▁▂▁▁▂▁▂▁▁▂▂▂
wandb:          G3_Acc ▄▇██▆█▁▆▃▁▃▆▇█▄▇██▃█▄▆▅▅▇█▇███▆▇▇▇▇██▆▆█
wandb:  Total Accuracy ▅▆▇▇▅█▂▅▃▁▅█▇▇▄▆▇▇▃▇▄▅▄▄▆▇▇▇▇▇▅▆▇▇▇▇▇▆▅█
wandb: Validation Loss ▁▃▄▅▅▅▆▆▇▇█▇▇▇█▇▇▇█▇█████▇██▇███████████
wandb: 
wandb: Run summary:
wandb:          G1_Acc 0.0
wandb:          G2_Acc 75.90361
wandb:          G3_Acc 57.57576
wandb:  Total Accuracy 60.29851
wandb: Validation Loss 2.72341
wandb: 
wandb: 🚀 View run microsoft/resnet-18 at: https://wandb.ai/pietro-caforio-politecnico-di-milano/unimodal_ct_training/runs/6m6gmdvs
wandb: ⭐️ View project at: https://wandb.ai/pietro-caforio-politecnico-di-milano/unimodal_ct_training
wandb: Synced 5 W&B file(s), 0 media file(s), 0 artifact file(s) and 0 other file(s)
wandb: Find logs at: ./wandb/run-20240916_113810-6m6gmdvs

### Resnet-34

In [17]:

model = ResNetForImageClassification.from_pretrained('microsoft/resnet-34')
model.classifier[-1] = nn.Linear(model.classifier[-1].in_features, UnimodalCTDataset.num_classes) #Adjusting the final layer to the unimodal number of classes

config.json:   0%|          | 0.00/69.5k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/87.3M [00:00<?, ?B/s]

In [18]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

ResNetForImageClassification(
  (resnet): ResNetModel(
    (embedder): ResNetEmbeddings(
      (embedder): ResNetConvLayer(
        (convolution): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        (normalization): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (activation): ReLU()
      )
      (pooler): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    )
    (encoder): ResNetEncoder(
      (stages): ModuleList(
        (0): ResNetStage(
          (layers): Sequential(
            (0): ResNetBasicLayer(
              (shortcut): Identity()
              (layer): Sequential(
                (0): ResNetConvLayer(
                  (convolution): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
                  (normalization): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
                  (activation): ReLU()
           

In [19]:
config={
    "learning_rate": 0.0001,
    "architecture": "microsoft/resnet-34",
    "epochs": 70,
    "weight_decay": 0.001,
    "reduce_lr_factor": 0.25,
    "patience": 20
    }
train(model, config, run_name = config["architecture"])


wandb: wandb version 0.18.0 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.17.7
wandb: Run data is saved locally in /kaggle/working/wandb/run-20240916_115607-scmcn0u2
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run microsoft/resnet-34
wandb: ⭐️ View project at https://wandb.ai/pietro-caforio-politecnico-di-milano/unimodal_ct_training
wandb: 🚀 View run at https://wandb.ai/pietro-caforio-politecnico-di-milano/unimodal_ct_training/runs/scmcn0u2


Epoch 1, Loss: 0.10334118311746149
Validation Loss: 1.517634645739401, Total Accuracy: 69.55%
Accuracy per class - G1: 0.00%, G2: 78.92%, G3: 77.27%
Epoch 2, Loss: 0.002122113253992413
Validation Loss: 1.6640042288609864, Total Accuracy: 71.94%
Accuracy per class - G1: 0.00%, G2: 76.51%, G3: 86.36%
Epoch 3, Loss: 0.0011614395067833054
Validation Loss: 1.6880409655700945, Total Accuracy: 72.24%
Accuracy per class - G1: 0.00%, G2: 77.71%, G3: 85.61%
Epoch 4, Loss: 0.004029440140180264
Validation Loss: 1.6464744794436477, Total Accuracy: 70.45%
Accuracy per class - G1: 0.00%, G2: 79.52%, G3: 78.79%
Epoch 5, Loss: 0.025211270468490612
Validation Loss: 1.8442254994403233, Total Accuracy: 57.91%
Accuracy per class - G1: 0.00%, G2: 75.30%, G3: 52.27%
Epoch 6, Loss: 0.00917614627623051
Validation Loss: 1.7093138015797278, Total Accuracy: 69.85%
Accuracy per class - G1: 0.00%, G2: 75.30%, G3: 82.58%
Epoch 7, Loss: 0.010344695854251363
Validation Loss: 1.8268792645963416, Total Accuracy: 72.24%


wandb:                                                                                
wandb: 
wandb: Run history:
wandb:          G1_Acc ▁▁▁▁▁▁▁▁▁▂█▂▇▄▇▄▅▆▄▄▄▄▅▅▅▅▅▄▄▃▅▂▂▁▂▄▅▅▅▄
wandb:          G2_Acc █▇█▇█▇██▇▁▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇
wandb:          G3_Acc ▅█▆▇█▄▆▂▆▇▃▂▄▃▃▃▃▃▂▁▂▂▂▁▁▂▄▂▂▄▄▄▄▁▁▁▂▂▄▂
wandb:  Total Accuracy ▇█▇▇█▆▇▅▇▁▇▅▇▆▇▆▆▆▆▅▆▆▆▆▆▆▆▆▆▆▇▆▆▅▅▅▆▆▇▆
wandb: Validation Loss ▁▂▂▂▃▄▄▄▄▃▄▅▅▆▅▆▆▅▆▆▅▅▅▆▅▆▆▆▆▆▄▆▇██▆▅▆▅▇
wandb: 
wandb: Run summary:
wandb:          G1_Acc 10.81081
wandb:          G2_Acc 75.3012
wandb:          G3_Acc 67.42424
wandb:  Total Accuracy 65.07463
wandb: Validation Loss 2.63184
wandb: 
wandb: 🚀 View run microsoft/resnet-34 at: https://wandb.ai/pietro-caforio-politecnico-di-milano/unimodal_ct_training/runs/scmcn0u2
wandb: ⭐️ View project at: https://wandb.ai/pietro-caforio-politecnico-di-milano/unimodal_ct_training
wandb: Synced 5 W&B file(s), 0 media file(s), 0 artifact file(s) and 0 other file(s)
wandb: Find logs at: ./wandb/run-20240916_115607-scmc